- 첨부하는 "sonar.all-data.csv" 파일은 **실린더모양의 암석(R)에서 튕겨져 나온 음파신호**와  **실린더 모양의 금속(M)에서 튕겨져 나온 음파신호**에 대한 정보를 담은 파일
- 이 csv 파일에는 **header가 없으며 208개의 샘플**에 대한 정보를 담고 있다. (처음부터 60개의 열은 0.1부터 1.0사이의 값을 갖는 특성이 저장되어 있고, 마지막 열에는 암석(R)인지 금속(M)인지가 저장되어 있다.)

쥬피터노트북에서 csv파일을 읽어들여 특성을 담은 X와 레이블을 담은 Y로 데이터 샘플을 만들고 아래의 작업을 수행한 후 쥬피터노트북 파일(ipynb파일)의 제목을 자신의 학번으로 만들어 12월17일 금요일 자정까지 제출하시오.

- 주어진 데이터 샘플을 이용하여 주어진 특성벡터(60차원)에 대해 R인지 M인지를 판별하는 3가지 모델을 **교차검증을 이용한 파라미터 설정을 포함하여 학습**시키고, **'accuracy'에 대한 cross_val_score**를 각각 구하시오.

# 특성벡터와 레이블벡터 만들기

### csv 파일 읽어들여 X와 Y를 만들기

In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
file_dir = 'C:/Users/82108/Desktop/dldusrud/3학년 2학기/ML2021/'

In [3]:
data_sonar = pd.read_csv(file_dir + 'sonar.all-data.csv', header=None)

In [4]:
data_sonar.head()

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [5]:
# 특성을 담은 X와 레이블을 담은 Y
X = data_sonar.iloc[:,:60]
y = data_sonar.iloc[:,[-1]]

In [6]:
# 특성을 정규화하기
# from sklearn.preprocessing import MinMaxScaler

# X =  MinMaxScaler().fit_transform(X.astype(np.float64))

In [7]:
# 특성을 표준화하기
from sklearn.preprocessing import StandardScaler

X =  StandardScaler().fit_transform(X.astype(np.float64))

In [8]:
# y를 문자가 아닌 숫자로 바꾸기 => R : 1, M : 0
y = (y == "R").astype(np.int)

In [9]:
# 전체 데이터 샘플의 20%는 테스트 샘플로 두고(test_size=0.2, seed=7) 10 fold 교차검증을 이용하여 파라미터 설정
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=7)

# [ Logistic Regression ]

### GridSearchCV를 이용하여 교차검증으로 적절한 하이퍼파라미터 선택
- 주요 파라미터 (C)
    - C값 (기본값 = 1)
    - C값이 작으면 Penalty 강해짐 (단순모델)
    - C값이 크면 Penalty 약해짐
    - 보통 로그스케일로 지정(10배씩) = 0.01, 0.1, 1, 10
- penalty
    - L2 : Ridge, 일반적으로 사용 (default)
    - L1 : Lasso, 변수가 많아서 줄여야 할 때 사용, 모델의 단순화 및 해석에 용이
    - Elasticnet : L1과 L2가 합쳐진 버전
    - none : 어떤 penalty도 추가되지 않음

* 참고한 사이트 :
    - https://wikidocs.net/16594
    - https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html?highlight=logisticregression#sklearn.linear_model.LogisticRegression

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# 파라미터 후보
params = {'C' : [0.001,0.01,0.1,1,10,100],
          'penalty':['l1','l2']}

# 그리드 서치 진행
grid_search_lr = GridSearchCV(LogisticRegression(), param_grid = params, scoring = 'accuracy', cv=10)

# param_grid의 하이퍼 파라미터를 순차적으로 학습/평가하기
grid_search_lr.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=LogisticRegression(),
             param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100],
                         'penalty': ['l1', 'l2']},
             scoring='accuracy')

In [11]:
# 최적의 파라미터 값 찾기
grid_search_lr.best_params_

{'C': 0.1, 'penalty': 'l2'}

In [12]:
# 최적 평균 정확도
grid_search_lr.best_score_

0.8011029411764706

In [13]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

best_lr = grid_search_lr.best_estimator_

cv10_scores = cross_val_score(best_lr, X_train, y_train, cv=10, scoring = 'accuracy')

print('\nlogistic regression의 10-겹 교차 검증 점수 : \n', cv10_scores)
print('\nlogistic regression의 10-겹 교차 검증 평균 점수 : ', cv10_scores.mean())

y_pred_train = best_lr.predict(X_train)
y_pred_test = best_lr.predict(X_test)

print(f"\n격자탐색 로지스틱 회귀 train dataset 정확도 : {accuracy_score(y_train,y_pred_train)}")
print(f"\n격자탐색 로지스틱 회귀 test dataset 정확도 : {accuracy_score(y_test,y_pred_test)}")


logistic regression의 10-겹 교차 검증 점수 : 
 [0.88235294 0.76470588 0.76470588 0.82352941 0.76470588 0.82352941
 0.6875     0.75       0.875      0.875     ]

logistic regression의 10-겹 교차 검증 평균 점수 :  0.8011029411764706

격자탐색 로지스틱 회귀 train dataset 정확도 : 0.8855421686746988

격자탐색 로지스틱 회귀 test dataset 정확도 : 0.7619047619047619


# [ Support Vector Machine ]

### GridSearchCV를 이용하여 교차검증으로 적절한 하이퍼파라미터 선택
- 주요 파라미터 (C)
    - C값 (기본값 = 1)
    - C값이 작으면 Penalty 강해짐 (단순모델)
    - C값이 크면 Penalty 약해짐
    - 보통 로그스케일로 지정(10배씩) = 0.01, 0.1, 1, 10
- kernel
SVM에서 특성벡터를 직접 구하는 대신 커널 트릭을 사용하여 쌍대문제 해결
    - linear : 선형 커널
    - rbf : 가우시안 RBF(Radial Basis Function)
        - gamma : 데이터 샘플이 영향력을 행사하는 거리를 결정 (클수록 한 데이터 포인터들이 영향력을 행사하는 거리가 짧아지고, 낮을수록 커진다)
    - poly : 다항커널
        - gamma
- 참고한 사이트
    - https://woolulu.tistory.com/tag/grid%20search
    - https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [14]:
from sklearn.svm import SVC

# 파라미터 후보
values = [0.001, 0.01, 0.1, 1, 10, 100]
params = [{'kernel' : ['rbf'], 'C' : values , 'gamma' : values},
           {'kernel' : ['linear'], 'C' : values},
           {'kernel' : ['poly'], 'C' : values, 'gamma' : values}]

# 그리드 서치 진행
grid_search_svc = GridSearchCV(SVC(), param_grid = params, scoring = 'accuracy', cv = 10)

# param_grid의 하이퍼 파라미터를 순차적으로 학습/평가하기
grid_search_svc.fit(X_train,y_train)

GridSearchCV(cv=10, estimator=SVC(),
             param_grid=[{'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'kernel': ['rbf']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'kernel': ['linear']},
                         {'C': [0.001, 0.01, 0.1, 1, 10, 100],
                          'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
                          'kernel': ['poly']}],
             scoring='accuracy')

In [15]:
# 최적의 파라미터 값 찾기
grid_search_svc.best_params_

{'C': 10, 'gamma': 0.01, 'kernel': 'rbf'}

In [16]:
# 최적의 평균 정확도
grid_search_svc.best_score_

0.8485294117647058

In [17]:
best_svc = grid_search_svc.best_estimator_

cv10_scores = cross_val_score(best_svc, X_train, y_train, cv = 10, scoring = 'accuracy')

print('\nSupport vector machine의 10-겹 교차 검증 점수 : \n', cv10_scores)
print('\nSupport vector machine의 10-겹 교차 검증 평균 점수 : ', cv10_scores.mean())

y_pred_train = best_svc.predict(X_train)
y_pred_test = best_svc.predict(X_test)

print(f"\n격자탐색 Support vector machine train dataset 정확도 : {accuracy_score(y_train,y_pred_train)}")
print(f"\n격자탐색 Support vector machine test dataset 정확도 : {accuracy_score(y_test,y_pred_test)}")


Support vector machine의 10-겹 교차 검증 점수 : 
 [0.94117647 0.70588235 0.94117647 0.88235294 1.         0.76470588
 0.6875     0.875      0.75       0.9375    ]

Support vector machine의 10-겹 교차 검증 평균 점수 :  0.8485294117647058

격자탐색 Support vector machine train dataset 정확도 : 1.0

격자탐색 Support vector machine test dataset 정확도 : 0.9285714285714286


# [ Decision Tree Classifier ]

### GridSearchCV를 이용하여 교차검증으로 적절한 하이퍼파라미터 선택
- max_leaf_nodes : 리프노드의 최대 개수
- min_samples_leaf : 리프노드가 가지고 있어야 할 최소 훈련 샘플의 개수
- max_depth : 결정트리의 최대깊이
- criterion
    - gini : 지니불순도
    - entropy : 엔트로피불순도

In [41]:
from sklearn.tree import DecisionTreeClassifier

# 파라미터 후보
params = {
    'criterion' : ['gini','entropy'],
    'max_depth' : range(1,10),
    'min_samples_leaf': range(15,20),
    'max_leaf_nodes': range(2, 50)}

# 그리드 서치 진행
grid_search_tr = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid = params, scoring = 'accuracy', cv=10)

# param_grid의 하이퍼 파라미터를 순차적으로 학습/평가하기
grid_search_tr.fit(X_train, y_train)

GridSearchCV(cv=10, estimator=DecisionTreeClassifier(random_state=42),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 10),
                         'max_leaf_nodes': range(2, 50),
                         'min_samples_leaf': range(15, 20)},
             scoring='accuracy')

In [43]:
# 최적의 파라미터 값 찾기
grid_search_tr.best_params_

{'criterion': 'gini',
 'max_depth': 1,
 'max_leaf_nodes': 2,
 'min_samples_leaf': 15}

In [44]:
# 최적의 평균 정확도
grid_search_tr.best_score_

0.6878676470588235

In [45]:
best_tr = grid_search_tr.best_estimator_

cv10_scores = cross_val_score(best_tr, X_train, y_train, cv=10, scoring = 'accuracy')

print('\nDecision tree의 10-겹 교차 검증 점수 : \n', cv10_scores)
print('\nDecision tree의 10-겹 교차 검증 평균 점수 : ', cv10_scores.mean())

y_pred_train = best_tr.predict(X_train)
y_pred_test = best_tr.predict(X_test)

print(f"\n격자탐색 Decision tree train dataset 정확도 : {accuracy_score(y_train,y_pred_train)}")
print(f"\n격자탐색 Decision tree test dataset 정확도 : {accuracy_score(y_test,y_pred_test)}")


Decision tree의 10-겹 교차 검증 점수 : 
 [0.64705882 0.70588235 0.70588235 0.70588235 0.58823529 0.58823529
 0.8125     0.5625     0.6875     0.875     ]

Decision tree의 10-겹 교차 검증 평균 점수 :  0.6878676470588235

격자탐색 Decision tree train dataset 정확도 : 0.7469879518072289

격자탐색 Decision tree test dataset 정확도 : 0.8095238095238095
